In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

## 整理銷售記錄
1. 以發票號碼為一筆交易記錄
2. 只留下交易產品，對於購物籃分析來說，計算只有機率，所以數量不需要

### 首先先看全年的銷售筆數，期間(2021/01/01~2022/03/31)
- 以發票號碼為購物車的話，總筆數會較少

### 讀取和從原始銷售數據萃取

In [3]:
sales = pd.read_excel(r"C:\Users\kc.hsu\Desktop\2021年銷售實績.xlsx")

In [11]:
sales.to_csv(r"C:\Users\kc.hsu\Desktop\2021年銷售實績.csv", index=False, encoding="utf8")

In [6]:
sales.columns = sales.columns.str.strip()
tbr = sales[sales["中計"].str.contains("TBR|LSR2")] #拆出TBR數據
psr = sales[~sales["中計"].str.contains("TBR|LSR2")] #拆出PSR數據

### Reshape TBR數據

In [12]:
# 要將後面數值能夠符合True False的function
def encode_data(data):
    if data <= 0:
        return 0
    else:
        return 1

In [15]:
whole_order = tbr[["發票號碼", "物料說明"]]

In [21]:
whole_order = whole_order.groupby(["發票號碼", "物料說明"])["發票號碼"].count().unstack().fillna(0)

In [22]:
whole_order = whole_order.applymap(encode_data)

In [23]:
whole_order

物料說明,1000R20 147K 16 R156Z 泰,1100R20 16 150K M840Z 泰,1100R20 16P 150K 1R156Z 泰,1100R20 16P R156Z 150K 泰,11R22.5 148L R156AZ 泰,1200R20 154K 18 M840BZ 泰,129M 9.5R17.5 R294Z 泰,12R22.5 152L 18P R118Z 泰,12R22.5 16 152L D600Z 泰,12R22.5 16 152L F400CZ 泰,12R22.5 16P 152L M729Z 泰,12R225 152L 16 R150AZ 日,12R225 152L 18 R150AZ 中,12R225 152L R156AZ 泰,12R225 16P 152L R150AZ 泰,12R225 16P 152L R224FZ 泰,1300R20 164F 20 VETZ 日,1400R20 22P 164F R180BZ 日,205/75R175 12 R227CZ 124M 日,205/75R175 123M R294Z 日,215/75 R175 126M R156Z 泰,215/75R175 124L R294Z 日,215/75R175 135J 16 R184GZ 日,225/70R19.5 130J M810Z 日,225/75R175 128M R294Z 日,225/90R17.5 127L R225Z 日,225/90R175 127L M810Z 日,235/75R 175 R294Z 132M 日,235/75R175 132M R156Z 泰,235/75R175 R156Z 132M 泰,245/70R17.5 B43J R168Z 日,245/70R19.5 136J G623Z 日,245/70R19.5 136M 14P R294Z 日,255/70R225 B40M R294Z 日,265/70R19.5 140M 14 R294Z 泰,275/70R22.5 148J R162AZ 泰,275/70R22.5 B50J UAP1Z 泰,275/70R225 B48J 18 R192MZ 中,275/70R225 B48J 18 R208AZ 中,285/70R 195 R184Z 148J 日,285/70R19.5 148K R184Z 日,295/80R22.5 152M 18 M778Z 中,295/80R22.5 152M 18 R150EZ 中,295/80R22.5 152M M798Z 泰,295/80R225 152M R150Z 日,295/80R225 152M 18 R102Z 中,295/80R225 152M R150Z 泰,295/80R225 152M R156AZ 泰,295/80R225 152M R294Z 日,295/80R225 B52J R192FZ 日,315/60R225 16P 152L R227CZ 日,315/80R22.5 154M D600BZ 泰,315/80R22.5 18P B54M R150Z 泰,315/80R22.5 B54M F400EZ 泰,315/80R225 154K G515Z 日,315/80R225 154L M805Z 泰,315/80R225 154M M749CZ 泰,315/80R225 18P B54M R150Z 日,315/80R225 B54M 18 M778Z 中,315/80R225 B54M 18 R150BZ 中,315/80R225 B54M 18 R150Z 泰,315/80R225 B54M F400EZ 泰,315/80R225 B54M G582BZ 泰,315/80R225 B54M M729UZ 泰,315/80R225 B54M R294Z 泰,315/80R225 B56K M840BZ 泰,385/65R225 18 B60K R179Z 泰,385/65R225 18 B60K R249Z 泰,385/65R225 B60K R168AZ 日,385/65R225 B60K R168AZ 泰,750 R 16 14P R225Z 日,825 R 16 14P R225Z 日,9.5R17.5 129M R294Z 日,900R20 14 G610Z 日,900R20 140K 14 G580BZ 泰
發票號碼,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HZ06278751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HZ06278753,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HZ06278762,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
HZ06278768,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HZ06278769,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZA17240698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
ZA17240706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
ZA17240716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0


### 建立apriori分析模組
- 參數：最小支持度設定在5%，太低的話可能組合太多會模糊視聽，以目前的情況來說，設定5%還不至於太少

In [34]:
freq_itemsets = apriori(whole_order, min_support=0.05, use_colnames=True)

In [30]:
freq_itemsets.sort_values("support", ascending=False)

,support,itemsets
16,0.242052,(9.5R17.5 129M R294Z 日)
9,0.213887,(315/80R225 B54M 18 R150Z 泰)
15,0.156999,(825 R 16 14P R225Z 日)
14,0.151422,(750 R 16 14P R225Z 日)
11,0.140825,(315/80R225 B54M M729UZ 泰)
7,0.139710,(295/80R225 152M R150Z 泰)
12,0.112660,(315/80R225 B54M R294Z 泰)
13,0.100390,(385/65R225 B60K R168AZ 泰)
8,0.091746,(315/80R225 18P B54M R150Z 日)
25,0.087563,"(9.5R17.5 129M R294Z 日, 825 R 16 14P R225Z 日)"


### 建立關聯規則

In [36]:
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1.2)

In [38]:
rules.sort_values(["confidence", "lift"], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(295/80R225 152M R150Z 泰),(315/80R225 B54M 18 R150Z 泰),0.139710,0.213887,0.079197,0.566866,2.650303,0.049315,1.814942
15,(825 R 16 14P R225Z 日),(9.5R17.5 129M R294Z 日),0.156999,0.242052,0.087563,0.557726,2.304156,0.049561,1.713753
7,(315/80R225 B54M M729UZ 泰),(315/80R225 B54M 18 R150Z 泰),0.140825,0.213887,0.077663,0.551485,2.578391,0.047542,1.752702
13,(750 R 16 14P R225Z 日),(9.5R17.5 129M R294Z 日),0.151422,0.242052,0.077942,0.514733,2.126535,0.041290,1.561919
9,(315/80R225 B54M R294Z 泰),(315/80R225 B54M 18 R150Z 泰),0.112660,0.213887,0.056470,0.501238,2.343466,0.032373,1.576127
10,(750 R 16 14P R225Z 日),(825 R 16 14P R225Z 日),0.151422,0.156999,0.071528,0.472376,3.008773,0.047755,1.597729
11,(825 R 16 14P R225Z 日),(750 R 16 14P R225Z 日),0.156999,0.151422,0.071528,0.455595,3.008773,0.047755,1.558725
3,(295/80R225 152M R150Z 泰),(315/80R225 B54M M729UZ 泰),0.139710,0.140825,0.057446,0.411178,2.919768,0.037771,1.459141
2,(315/80R225 B54M M729UZ 泰),(295/80R225 152M R150Z 泰),0.140825,0.139710,0.057446,0.407921,2.919768,0.037771,1.452998
0,(315/80R225 B54M 18 R150Z 泰),(295/80R225 152M R150Z 泰),0.213887,0.139710,0.079197,0.370274,2.650303,0.049315,1.366133


In [39]:
rules.to_excel(r"C:\Users\kc.hsu\Desktop\購物籃分析.xlsx", index=False)

In [48]:
pcode_list = tbr[["物料", "物料說明"]]

In [62]:
pcode_list.drop_duplicates().reset_index(drop=True)

,物料,物料說明
0,LSR03733,215/75R175 124L R294Z 日
1,TBR02508,295/80R225 152M R294Z 日
2,TBR0J247,295/80R225 152M R150Z 泰
3,LSR01270,825 R 16 14P R225Z 日
4,LSR05127,205/75R175 123M R294Z 日
...,...,...
73,TBR0J299,1200R20 154K 18 M840BZ 泰
74,TBR06381,285/70R 195 R184Z 148J 日
75,TBR0J642,275/70R22.5 B50J UAP1Z 泰
76,LSR0L018,235/75R175 132M R156Z 泰


In [63]:
pcode_list["產地"] = pcode_list["物料說明"].str.split(" ")

C:\Users\kc.hsu\AppData\Local\Temp/ipykernel_10492/2948416411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pcode_list["產地"] = pcode_list["物料說明"].str.split(" ")


In [70]:
pcode_list["產地"] = pcode_list["產地"].map(lambda x: x[-1])

C:\Users\kc.hsu\AppData\Local\Temp/ipykernel_10492/3028645973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pcode_list["產地"] = pcode_list["產地"].map(lambda x: x[-1])


In [73]:
pcode_list["物料說明"] = pcode_list["物料說明"].map(lambda x: x[:-1])

C:\Users\kc.hsu\AppData\Local\Temp/ipykernel_10492/1183869969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pcode_list["物料說明"] = pcode_list["物料說明"].map(lambda x: x[:-1])


In [75]:
tbr

,組織,出貨點,所名,訂單建立日,訂單文件日,訂單建立時,交貨文件日,計劃出貨日,實際出貨日,交單建立時,請款建立日,請款日期,請款建立時,訂單建立人,交單建立人,請款建立人,訂貨原因,訂單備註,產品階層,採購單類型,免費,訂單類型,交貨類型,請款類型,訂單參考,項次參考,訂單單號,訂單項次,上層項,交貨單號,交貨項目,出貨過帳,請款文件,發票號碼,請款項目,買方,送貨方,彙總客戶,業務員,買方名,送貨方名,彙總戶名,業務員名,送貨地郵遞號,送貨城市,送貨地區,通路,大胎,小胎,付款條件,付款方法,排序,物料群組,分類,物料,物料說明,花紋別,NEXTRY,Premium,UHP,P+U,B,B.1,G,F,國別,品牌,吋別,構造,膠重,中計,吋別點數,客戶物料,折價券號碼,訂單數量,訂單單價,未稅合計,含稅合計,交貨數量,未稅出貨,含稅出貨,發票數量,發票單價,發票未稅,稅額,發票合計,銷售數量,銷貨單價,未稅銷貨
32,FCC1,FF11,鶯歌,2021-01-04,2021-01-04,09:00:46,2021-01-04,2021-01-04,2021-01-04,09:01:32,2021-01-31,2021-01-31,20:49:29,F11SAS01,F11SAS01,F11SAS01,NaN,NaN,NaN,NaN,NaN,ZSO1,ZF02,ZF2,NaN,NaN,103953406,10,NaN,280197305,10,NaN,90767809.0,KT16641993,10.0,C1C10413,C1C10413,C1C10413,C11,勇和,勇和,勇和,梁龍,236.0,新北市,土城區,H,3,3,W030,C,4,TLSRJ,進口,LSR03733,215/75R175 124L R294Z 日,R294Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP,BS,175,T,12.479,LSR2,NaN,NaN,NaN,10,6239.0,59419.0,62390.0,10,59419.0,62390.0,10.0,6239.0,59419.0,2971.0,62390.0,10,6239.0,59419.0
33,FCC1,FF11,鶯歌,2021-01-04,2021-01-04,09:01:36,2021-01-04,2021-01-04,2021-01-04,09:01:45,2021-01-31,2021-01-31,20:49:29,F11SAS01,F11SAS01,F11SAS01,NaN,NaN,NaN,NaN,NaN,ZSO1,ZF02,ZF2,NaN,NaN,103953409,10,NaN,280197308,10,NaN,90767812.0,KT16641996,10.0,C1C10658,C1C10658,C1C10658,C11,百樂達,百樂達,百樂達,梁龍,333.0,桃園縣,龜山,H,1,2,W030,C,4,TLSRJ,進口,LSR03733,215/75R175 124L R294Z 日,R294Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP,BS,175,T,12.479,LSR2,NaN,NaN,NaN,6,6239.0,35651.0,37434.0,6,35651.0,37434.0,6.0,6239.0,35651.0,1783.0,37434.0,6,6239.0,35651.0
35,FCC1,FF11,鶯歌,2021-01-04,2021-01-04,09:01:47,2021-01-04,2021-01-04,2021-01-04,09:01:58,2021-01-31,2021-01-31,20:49:29,F11SAS01,F11SAS01,F11SAS01,NaN,還 揚明,NaN,NaN,NaN,ZSO1,ZF02,ZF2,NaN,NaN,103953412,10,NaN,280197311,10,NaN,90767812.0,KT16641996,11.0,C1C10658,C1C10658,C1C10658,C11,百樂達,百樂達,百樂達,梁龍,333.0,桃園縣,龜山,H,1,2,W030,C,4,TLSRJ,進口,LSR03733,215/75R175 124L R294Z 日,R294Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP,BS,175,T,12.479,LSR2,NaN,NaN,NaN,1,6239.0,5942.0,6239.0,1,5942.0,6239.0,1.0,6239.0,5942.0,297.0,6239.0,1,6239.0,5942.0
151,FCC1,FF11,鶯歌,2021-01-04,2021-01-04,11:11:33,2021-01-04,2021-01-04,2021-01-04,11:13:42,2021-01-31,2021-01-31,20:49:29,F11SAS01,F11SAS01,F11SAS01,NaN,NaN,NaN,NaN,NaN,ZSO1,ZF02,ZF2,NaN,NaN,103954081,10,NaN,280197966,10,NaN,90767812.0,KT16641996,15.0,C1C10658,C1C10658,C1C10658,C11,百樂達,百樂達,百樂達,梁龍,333.0,桃園縣,龜山,H,1,2,W030,C,1,TTBR,進口,TBR02508,295/80R225 152M R294Z 日,R294Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP,BS,225,T,27.594,TBR3,NaN,NaN,NaN,2,11093.0,21128.0,22185.0,2,21128.0,22185.0,2.0,11092.0,21128.0,1056.0,22184.0,2,11092.0,21128.0
152,FCC1,FF11,鶯歌,2021-01-04,2021-01-04,11:11:33,2021-01-04,2021-01-04,2021-01-04,11:13:42,2021-01-31,2021-01-31,20:49:29,F11SAS01,F11SAS01,F11SAS01,NaN,NaN,NaN,NaN,NaN,ZSO1,ZF02,ZF2,NaN,NaN,103954081,20,NaN,280197966,20,NaN,90767812.0,KT16641996,20.0,C1C10658,C1C10658,C1C10658,C11,百樂達,百樂達,百樂達,梁龍,333.0,桃園縣,龜山,H,1,2,W030,C,1,TTBR,進口,TBR0J247,295/80R225 152M R150Z 泰,R150Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TH,BS,225,T,27.991,TBR3,NaN,NaN,NaN,1,10484.0,9985.0,10484.0,1,9985.0,10484.0,1.0,10484.0,9985.0,499.0,10484.0,1,10484.0,9985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387980,FCC1,FF74,屏東,2022-03-31,2022-03-31,12:06:50,2022-03-31,2022-03-31,2022-03-31,12:07:21,2022-03-31,2022-03-31,22:54:55,F51SAS03,F51SAS03,F51SAS02,NaN,NaN,NaN,NaN,NaN,ZSO1,ZF02,ZF2,NaN,NaN,104260430,10,NaN,280526459,10,NaN,90797099.0,ZA17240698,369.0,N1N1B040,N1N1B040,N1N1B040,V02,宏通汽車,宏通汽車,宏通汽車,郭昭賢,928.0,屏東縣,東港,H,1,1,W030,C,4,TLSRJ,進口,LSR03459,9.5R17.5 129M R294Z 日,R294Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP,BS,175,T,16.286,LSR2,NaN,NaN,NaN,10,6938.0,66080.0,69384.0,10,66080.0,69384.0,10.0,6938.0,66080.0,3304.0,69384.0,10,6938.0,66080.0
387981,FCC1,FF

In [124]:
pcode_list = pd.read_clipboard()

In [125]:
pcode_list = pcode_list[["Main PCODE", "size", "pattern", "p_origin"]]

In [126]:
pcode_list["描述"] = pcode_list["size"] + ", " + pcode_list["pattern"]

In [127]:
pcode_list["產地"] = pcode_list["Main PCODE"].map(lambda x: "泰" if x.__contains__("0J") or x.__contains__("0L") else "中" if x.endswith("F") or x.__contains__("0H") else "日")

In [128]:
pcode_list = pcode_list[["Main PCODE", "描述", "產地"]].drop_duplicates()

In [134]:
pcode_list.to_excel(r"C:\Users\kc.hsu\Desktop\產地_描述_PCODE對照表.xlsx", index=False)

In [133]:
pcode_list = pcode_list.drop_duplicates(subset=["描述", "產地"], keep="last")

In [190]:
rescue = pd.read_clipboard()

In [170]:
cus_data = pd.read_clipboard()

In [174]:
cus_data = cus_data.reset_index()

In [175]:
cus_data.columns = cus_data.columns.str.strip()

In [176]:
cus_data.columns

Index(['客戶編號', '客戶簡稱', '客戶名稱', '區號', '住址', '地址註記', '電話', '統一編號', '調節科目', '條件',
       '方式', '建立日期', '銷售群組', '販社通路', '大胎契約', '小胎契約', '業績彙總業務員', '定價程序', '區號.1',
       '住址(發票收受人)', '總信用額度', '個別控制區域額度', '幣別', '銀行碼', '銀行帳戶', '銀行戶名', '銀行名稱',
       '資料行'],
      dtype='object')

In [178]:
cus_dict = dict(zip(cus_data["電話"], cus_data["客戶編號"]))

In [188]:
def find_cus_code(data):
    for item in cus_dict.keys():
        if str(item).__contains__(data):
            return cus_dict[item]

In [193]:
rescue["據點電話"] = rescue["據點電話"].astype(str)

In [196]:
rescue["BS客戶代號"] = rescue["據點電話"].map(find_cus_code)

In [198]:
reverse_check = dict(zip(cus_data["客戶編號"], cus_data["客戶名稱"]))

In [200]:
rescue["客戶主檔上客戶名稱"] = rescue["BS客戶代號"].map(reverse_check)

In [204]:
rescue["驗證欄位"] = rescue.apply(lambda x: "OK" if x["營業所名稱"] == x["客戶主檔上客戶名稱"] else "Need to check", axis=1)

In [206]:
rescue.to_excel(r"C:\Users\kc.hsu\Desktop\道路救援驗證資料.xlsx", index=False)